In [1]:
import numpy as np
from tqdm import tqdm_notebook

from qucumber.utils import unitaries

import import_ipynb
from kron_mv_python_benchmark import kron_mv_low_mem

importing Jupyter notebook from kron_mv_python_benchmark.ipynb
593 ms ± 63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.7 ms ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
def generate_hilbert_space(size):
    dim = np.arange(2 ** size)
    space = (((dim[:, None] & (1 << np.arange(size)))) > 0)[:, ::-1]
    space = space.astype(int)
    return space

In [3]:
def convert_torch_cplx(tensor):
    real_part = tensor[0].cpu().detach().numpy()
    imag_part = tensor[1].cpu().detach().numpy()
    
    return real_part + (1j * imag_part)

In [4]:
unitary_dict = {k: convert_torch_cplx(v) for k, v in unitaries.create_dict().items()}

In [5]:
def rotate_psi(N, psi, basis, unitary_dict, vis):
    return kron_mv_low_mem(psi, *[unitary_dict[b] for b in basis])

In [6]:
def get_psi_from_sample(v, psi):
    index = int("".join(str(i) for i in v), base=2)
    return psi[index]

In [7]:
def get_samples_from_psi_indices(indices, num_sites):
    return (((indices[:, None] & (1 << np.arange(num_sites)))) > 0)[:, ::-1].astype(int)

In [8]:
from itertools import product
def gen_all_bases(unitary_dict, num_sites):
    local_bases = unitary_dict.keys()
    return list("".join(i) for i in product(local_bases, repeat=num_sites))

In [9]:
def gen_samples(num_samples, num_sites, psi, basis, unitary_dict, vis):
    psi_r = rotate_psi(num_sites, psi, basis, unitary_dict, vis)
    
    probs = (psi_r.conj() * psi_r).real
    
    indices = np.random.choice(len(probs), size=num_samples, p=probs)
    return get_samples_from_psi_indices(indices, num_sites)

In [10]:
def gen_data(num_sites, num_samples_per_basis, unitary_dict, seed):
    np.random.seed(seed)
    
    size = 2 ** num_sites

    vis = generate_hilbert_space(num_sites)

    psi = np.random.randn(size) + 1j*np.random.randn(size) # gen rand wvfn
    psi /= np.sqrt((np.dot(psi.conj(), psi)).real)         # normalize
    
    all_bases = gen_all_bases(unitary_dict, num_sites)
    
    tr_bases = np.zeros((len(all_bases), num_sites), dtype=str)
    samples = np.zeros((len(all_bases), num_samples_per_basis, num_sites), dtype=int)
    
    for i, basis in enumerate(tqdm_notebook(all_bases)):
        tr_bases[i, :] = np.array(list(basis))
        samples[i, :, :] = gen_samples(num_samples_per_basis, num_sites, psi, basis, unitary_dict, vis)
        
    tr_bases = np.repeat(tr_bases[:, None, :], num_samples_per_basis, axis=1).reshape(-1, num_sites)
    samples = samples.reshape(-1, num_sites)
    
    all_bases = np.array(list(map(list, all_bases)))
    
    return all_bases, tr_bases, samples, psi

In [11]:
all_bases, tr_bases, samples, psi = gen_data(8, 200, unitary_dict, 161)

KeyboardInterrupt: 

In [ ]:
np.savetxt("8sites/all_bases.txt", all_bases, fmt='%s')
np.savetxt("8sites/tr_bases.txt", tr_bases, fmt='%s')
np.savetxt("8sites/samples.txt", samples)
np.savetxt("8sites/psi.txt", psi)